In [1]:
%run ../Python_files/util_data_storage_and_load.py

In [2]:
%run ../Python_files/load_dicts.py

In [3]:
%run ../Python_files/util.py

In [4]:
road_seg_inr_capac = zload('../temp_files/road_seg_inr_capac.pkz')

In [5]:
# load tmc-day-ave_speed data for AM peak of April
tmc_day_speed_dict_Apr_AM = zload('../temp_files/Apr_AM/tmc_day_speed_dict.pkz')

# load tmc-day-ave_speed data for MD of April
tmc_day_speed_dict_Apr_MD = zload('../temp_files/Apr_MD/tmc_day_speed_dict.pkz')

# load tmc-day-ave_speed data for PM peak of April
tmc_day_speed_dict_Apr_PM = zload('../temp_files/Apr_PM/tmc_day_speed_dict.pkz')

# load tmc-day-ave_speed data for NT of April
tmc_day_speed_dict_Apr_NT = zload('../temp_files/Apr_NT/tmc_day_speed_dict.pkz')

In [7]:
tmc_day_speed_dict_Apr_AM['129N0410818'].ave_speed()

60.150640412923032

In [9]:
tmc_day_capac_flow_minute_dict = {}

for i in range(len(road_seg_inr_capac.tmc)):
    for day in range(31)[1:]:
        tmc = road_seg_inr_capac.tmc[i]
        road_num = road_seg_inr_capac.road_num[i]
        shape_length = road_seg_inr_capac.shape_length[i]
        day = day
        AB_AM_capac = road_seg_inr_capac.AB_AM_capac[i]
        AB_MD_capac = road_seg_inr_capac.AB_MD_capac[i]
        AB_PM_capac = road_seg_inr_capac.AB_PM_capac[i]
        AB_NT_capac = road_seg_inr_capac.AB_NT_capac[i]
        AM_ave_speed = tmc_day_speed_dict_Apr_AM[tmc + str(day)].ave_speed()
        MD_ave_speed = tmc_day_speed_dict_Apr_MD[tmc + str(day)].ave_speed()
        PM_ave_speed = tmc_day_speed_dict_Apr_PM[tmc + str(day)].ave_speed()
        NT_ave_speed = tmc_day_speed_dict_Apr_NT[tmc + str(day)].ave_speed()
        AM_speed_minute = tmc_day_speed_dict_Apr_AM[tmc + str(day)].speed
        MD_speed_minute = tmc_day_speed_dict_Apr_MD[tmc + str(day)].speed
        PM_speed_minute = tmc_day_speed_dict_Apr_PM[tmc + str(day)].speed
        NT_speed_minute = tmc_day_speed_dict_Apr_NT[tmc + str(day)].speed
        tmc_day_capac_flow_minute = RoadSegInrCapacFlowMinute(tmc, road_num, shape_length, day, \
                                                  AB_AM_capac, AB_MD_capac, \
                                                  AB_PM_capac, AB_NT_capac, \
                                                  AM_ave_speed, MD_ave_speed, \
                                                  PM_ave_speed, NT_ave_speed, \
                                                  AM_speed_minute, MD_speed_minute, \
                                                  PM_speed_minute, NT_speed_minute)
        assert(len(tmc_day_capac_flow_minute.AM_flow_minute()) == 120)
        assert(len(tmc_day_capac_flow_minute.MD_flow_minute()) == 120)
        assert(len(tmc_day_capac_flow_minute.PM_flow_minute()) == 120)
        assert(len(tmc_day_capac_flow_minute.NT_flow_minute()) == 120)
        tmc_day_capac_flow_minute_dict[tmc + str(day)] = tmc_day_capac_flow_minute

In [75]:
#zdump(tmc_day_capac_flow_minute_dict, '../temp_files/tmc_day_capac_flow_minute_dict_Apr.pkz')

In [10]:
tmc_day_capac_flow_minute_dict['129N0437720'].NT_flow_minute

<bound method RoadSegInrCapacFlowMinute.NT_flow_minute of <__main__.RoadSegInrCapacFlowMinute object at 0x7f7e57823790>>

In [11]:
link_with_capac_list = list(zload('../temp_files/links_with_capac.pkz'))

In [12]:
import numpy as np

link_day_minute_Apr_dict = {}

for day in range(31)[1:]:
    i = 0
    for link_with_capac in link_with_capac_list:
        AM_flow = sum([tmc_day_capac_flow_minute_dict[tmc + str(day)].AM_flow() for tmc in link_with_capac.tmc_set])
        MD_flow = sum([tmc_day_capac_flow_minute_dict[tmc + str(day)].MD_flow() for tmc in link_with_capac.tmc_set])
        PM_flow = sum([tmc_day_capac_flow_minute_dict[tmc + str(day)].PM_flow() for tmc in link_with_capac.tmc_set])
        NT_flow = sum([tmc_day_capac_flow_minute_dict[tmc + str(day)].NT_flow() for tmc in link_with_capac.tmc_set])
        AM_flow_minute = list(sum([np.array(tmc_day_capac_flow_minute_dict[tmc + str(day)].AM_flow_minute()) \
                              for tmc in link_with_capac.tmc_set]))
        MD_flow_minute = list(sum([np.array(tmc_day_capac_flow_minute_dict[tmc + str(day)].MD_flow_minute()) \
                              for tmc in link_with_capac.tmc_set]))
        PM_flow_minute = list(sum([np.array(tmc_day_capac_flow_minute_dict[tmc + str(day)].PM_flow_minute()) \
                              for tmc in link_with_capac.tmc_set]))
        NT_flow_minute = list(sum([np.array(tmc_day_capac_flow_minute_dict[tmc + str(day)].NT_flow_minute()) \
                              for tmc in link_with_capac.tmc_set]))
        link_with_capac_new = Link_with_Free_Flow_Time_Minute(link_with_capac.init_node, link_with_capac.term_node, \
                                                       link_with_capac.tmc_set, \
                                                       link_with_capac.AM_capac, \
                                                       link_with_capac.MD_capac, \
                                                       link_with_capac.PM_capac, \
                                                       link_with_capac.NT_capac, \
                                                       link_with_capac.free_flow_time, \
                                                       link_with_capac.length, \
                                                       AM_flow, MD_flow, PM_flow, NT_flow, \
                                                       AM_flow_minute, MD_flow_minute, \
                                                       PM_flow_minute, NT_flow_minute)
        link_day_minute_Apr_dict['link_' + str(i) + '_' + str(day)] = link_with_capac_new
        i = i + 1

In [86]:
#zdump(link_day_minute_Apr_dict, '../temp_files/link_day_minute_Apr_dict.pkz')

In [13]:
link_day_minute_Apr_dict['link_0_10'].AM_flow

334585.81301312696

In [18]:
link_day_minute_Apr_dict_JSON = {}
for link_idx in range(24):
    for day in range(31)[1:]:   
        key = 'link_' + str(link_idx) + '_' + str(day)
        data = {'link_idx': link_idx, 'day': day, \
             'init_node': link_day_minute_Apr_dict[key].init_node, \
             'term_node': link_day_minute_Apr_dict[key].term_node, \
             'AM_capac': link_day_minute_Apr_dict[key].AM_capac, \
             'MD_capac': link_day_minute_Apr_dict[key].MD_capac, \
             'PM_capac': link_day_minute_Apr_dict[key].PM_capac, \
             'NT_capac': link_day_minute_Apr_dict[key].NT_capac, \
             'free_flow_time': link_day_minute_Apr_dict[key].free_flow_time, \
             'length': link_day_minute_Apr_dict[key].length, \
             'AM_flow': link_day_minute_Apr_dict[key].AM_flow, \
             'MD_flow': link_day_minute_Apr_dict[key].MD_flow, \
             'PM_flow': link_day_minute_Apr_dict[key].PM_flow, \
             'NT_flow': link_day_minute_Apr_dict[key].NT_flow, \
             'AM_flow_minute': link_day_minute_Apr_dict[key].AM_flow_minute, \
             'MD_flow_minute': link_day_minute_Apr_dict[key].MD_flow_minute, \
             'PM_flow_minute': link_day_minute_Apr_dict[key].PM_flow_minute, \
             'NT_flow_minute': link_day_minute_Apr_dict[key].NT_flow_minute}
        link_day_minute_Apr_dict_JSON[key] = data
        
import json

# Writing JSON data
with open('../temp_files/link_day_minute_Apr_dict_JSON.json', 'w') as json_file:
    json.dump(link_day_minute_Apr_dict_JSON, json_file)

In [20]:
with open('../temp_files/link_day_minute_Apr_dict_JSON.json', 'r') as json_file:
    link_day_minute_Apr_dict_JSON_ = json.load(json_file)

In [22]:
link_day_minute_Apr_dict_JSON_['link_3_9'] ['AM_flow_minute']

[666204.234109018,
 647170.0231859997,
 648481.9108126306,
 654093.970825819,
 661653.7549811001,
 657809.0235822904,
 651705.5350470361,
 652956.8175558159,
 660684.8868278614,
 646179.7236364048,
 639993.7802094242,
 658690.8956351925,
 662297.827966267,
 650096.2383474094,
 659868.2656340672,
 655959.8646669951,
 651252.9162960367,
 656025.4094720585,
 644421.2817657958,
 652717.4428806286,
 666801.6171790673,
 650792.0406505262,
 647896.3301050129,
 641101.4972620052,
 640549.8266441685,
 651770.2032300783,
 645063.9420626068,
 656546.7495404546,
 653949.1033054295,
 656548.8681227769,
 647311.8825359207,
 652269.0899956275,
 642730.0040780619,
 653939.0510149815,
 646931.5811752426,
 657454.6345466152,
 662708.9528040374,
 651317.115400684,
 652315.8951344035,
 654010.9025348632,
 663094.8160749042,
 650182.6561074846,
 646884.7046399973,
 640794.1420810363,
 645652.128694513,
 654282.4464408287,
 654302.7250131413,
 655351.9071667858,
 644940.2121503584,
 648994.6871268913,
 6599